In [56]:
import pandas as pd
import numpy as np
import groundhog.general.agsconversion as ghconverter
converter = ghconverter.AGSConverter("../data/raw/HKN_20190306_FNLM_HKN14-BH-SA_CPT_AGS4.0_V05_F.ags", encoding='utf8', errors='replace', removedoublequotes=True)

scpt_raw_data = converter.convert_ags_group("SCPT")
scpt_raw_data.describe()



,SCPT_DPTH [m],SCPT_RES [MN/m2],SCPT_FRES [kN/m2],SCPT_PWP2 [kN/m2],SCPT_FRR [%],SCPT_QT [MN/m2],SCPT_QNET [MN/m2],SCPT_BQ,SCPT_CIC1,SCPT_CISB,SCPT_NQC1,SCPT_NRF1,SCPT_CUW1 [kN/m3]
count,1624.000000,1624.000000,1531.000000,1602.000000,1531.000000,1624.000000,1624.000000,1624.000000,1414.000000,1414.000000,1414.000000,1414.000000,1.414000e+03
mean,42.973750,29.867746,250.821594,263.382834,0.884401,29.933949,29.087378,-0.006887,1.872765,1.503762,108.760537,0.878177,1.970000e+01
std,9.521547,11.171482,98.970419,194.740392,0.396529,11.171367,11.144544,0.033035,0.197418,0.159499,55.948195,0.406046,3.553971e-15
min,26.500000,0.252000,35.136000,-224.200000,0.393000,0.331000,-0.265000,-0.998400,1.520000,1.260000,15.000000,0.405000,1.970000e+01
25%,34.735000,23.846500,190.946500,171.200000,0.734000,23.884000,23.130000,-0.007600,1.750000,1.410000,73.725000,0.743000,1.970000e+01
50%,42.970000,30.703500,238.459000,307.600000,0.789000,30.762500,29.878500,-0.002500,1.860000,1.470000,91.950000,0.799000,1.970000e+01
75%,51.225000,35.989500,293.596500,405.850000,0.877000,36.062500,35.041500,-0.000800,1.960000,1.540000,127.275000,0.878750,1.970000e+01
max,59.420000,59.980000,669.044000,788.100000,3.697000,60.093000,58.947000,0.554900,2.940000,2.430000,311.400000,4.183000,1.970000e+01


In [57]:

# scpt seismic_cone_penetration_testing
scpt_named_data = scpt_raw_data.rename(columns={"SCPT_DPTH [m]": "depth", "SCPT_QT [MN/m2]": "corrected_con_resistance", "SCPT_FRES [kN/m2]": "skin_friction", "SCPT_PWP2 [kN/m2]": "pore_pressure"})
scpt_filtered_data = scpt_named_data[["depth", "corrected_con_resistance", "skin_friction", "pore_pressure"]]

scpt_filtered_data.describe()
scpt_filtered_data.head()


,depth,corrected_con_resistance,skin_friction,pore_pressure
0,26.50,0.718,NaN,NaN
1,26.52,1.758,NaN,265.1
2,26.54,2.690,NaN,262.2
3,26.56,2.892,51.795,260.9
4,26.58,3.367,68.160,260.8


In [58]:
scpt_data = scpt_filtered_data.dropna(subset=["corrected_con_resistance", "skin_friction"])

# Check
scpt_data["corrected_con_resistance"].isna().sum()
scpt_data["skin_friction"].isna().sum()

scpt_data.head()


,depth,corrected_con_resistance,skin_friction,pore_pressure
3,26.56,2.892,51.795,260.9
4,26.58,3.367,68.160,260.8
5,26.60,5.527,82.517,258.4
6,26.62,8.265,95.634,258.0
7,26.64,10.528,108.279,258.0


In [59]:

# Rf = fs /qt
# Rf = friction ratio
# fs = SCPT_FRES skin friction
# qt = SCPT_QT corrected penetration resistance
friction_ratio = scpt_data["skin_friction"] / scpt_data["corrected_con_resistance"]
scpt_data.insert(4, "friction_ratio", friction_ratio)
scpt_data.head()
# soil_data_friction_ratio_parameter = soil_data["SCPT_FRES", "SCPT_QT"]

,depth,corrected_con_resistance,skin_friction,pore_pressure,friction_ratio
3,26.56,2.892,51.795,260.9,17.909751
4,26.58,3.367,68.160,260.8,20.243540
5,26.60,5.527,82.517,258.4,14.929799
6,26.62,8.265,95.634,258.0,11.570962
7,26.64,10.528,108.279,258.0,10.284859


In [60]:
# Normalized Cone Resistance 𝑄𝑡 = (𝑞𝑡 − 𝜎𝑣0)/𝜎𝑣0
# Effective Overburden Stress 𝜎𝑣0= Σ(𝑧𝑖∙ 𝛾𝑖) − u 
# qt = corrected_con_resistance
# zi = depth
# 𝛾𝑖 = 10
# u = pore_pressure

stress = scpt_data["depth"].multiply(10) - scpt_data["pore_pressure"]
scpt_data.insert(5, "stress", stress, True)
scpt_data.head()

,depth,corrected_con_resistance,skin_friction,pore_pressure,friction_ratio,stress
3,26.56,2.892,51.795,260.9,17.909751,4.7
4,26.58,3.367,68.160,260.8,20.243540,5.0
5,26.60,5.527,82.517,258.4,14.929799,7.6
6,26.62,8.265,95.634,258.0,11.570962,8.2
7,26.64,10.528,108.279,258.0,10.284859,8.4
